In [1]:
# !cp CPPred/bin/feamodule/CTD.py CTD.py
# !cp CPPred/bin/feamodule/ProtParam.py ProtParam.py
# !cp CPPred/bin/feamodule/ORF.py ORF.py
# !cp CPPred/bin/feamodule/fickett.py fickett.py
# !cp CPPred/bin/feamodule/FrameKmer.py FrameKmer.py
# !cp CPPred/bin/feamodule/ORF_length.py ORF_length.py

In [2]:
!pip install Bio
!pip install biopython==1.75 --user

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 2.3MB 14.5MB/s eta 0:00:01
  Found existing installation: biopython 1.75
    Uninstalling biopython-1.75:
      Successfully uninstalled biopython-1.75
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 2.3MB 14.3MB/s eta 0:00:01
ERROR: bio 0.3.0 has requirement biopython>=1.78, but you'll have biopython 1.75 which is incompatible.


python CPPred.py 

-i ../data/Human_coding_RNA_test.fa 

-hex ../Hexamer/Human_Hexamer.tsv 

-r ../Human_Model/Human.range 

-mol ../Human_Model/Human.model 

-spe Human 

-o Human_coding.result

In [16]:
import os,sys

hex_file="CPPred/Hexamer/Human_Hexamer.tsv"
seq_file="CPPred/data/Human_coding_RNA_test.fa"

In [17]:
lines = open(hex_file).readlines()
lines[:5]

['hexamer\tcoding\tnoncoding\n',
 'GAACGT\t0.000125472613061\t6.29880112819e-05\n',
 'CTTCTT\t0.000299994100152\t0.000441227130881\n',
 'CACCCT\t0.000259819200042\t0.000355221278439\n',
 'GAACGG\t0.000195066081257\t6.15882776978e-05\n']

In [18]:
coding={}
noncoding={}
for line in lines:
    fields = line.split()
    if fields[0] == 'hexamer':continue           # 表头
    coding[fields[0]] = float(fields[1])
    noncoding[fields[0]] =  float(fields[2])

In [19]:
import Bio.SeqIO as Seq
seqs = Seq.parse(seq_file,'fasta')

for seq in seqs:
    print(seq);
    break;

ID: gi|1238415392|ref|NM_001354394.1|
Name: gi|1238415392|ref|NM_001354394.1|
Description: gi|1238415392|ref|NM_001354394.1| Homo sapiens diphosphoinositol pentakisphosphate kinase 1 (PPIP5K1), transcript variant 20, mRNA
Number of features: 0
Seq('ACCGCAAAGAAGTCGCGCTTTGATTGGGCCGCTGTCAGGTGTCGCTGCCTGCAG...GGA', SingleLetterAlphabet())


In [7]:
# import importlib
# importlib.reload(len)

In [20]:
tmp = open('test.f_svm','w')
feature = open('test.feature','w')

# species == "Human":
feature.write("\t".join(map(str,["#ID","ORF-integrity","ORF-coverage","Instability","T2","C0","PI","ORF-length","AC","T0","G0","C2","A4","G2","TG","A0","TC","G1","C3","T3","A1","GC","T1","G4","C1","G3","A3","Gravy","Hexamer","C4","AG","Fickett","A2","T4","C","G","A","T"]))+"\n")

type(feature)

_io.TextIOWrapper

In [21]:
import CTD
import ProtParam as PP
import fickett
import FrameKmer
import ORF_length as len

out_label = 1

for seq in seqs:
    seqid = seq.id
    A,T,G,C,AT,AG,AC,TG,TC,GC,A0,A1,A2,A3,A4,T0,T1,T2,T3,T4,G0,G1,G2,G3,G4,C0,C1,C2,C3,C4 = CTD.CTD(seq.seq)
    insta_fe,PI_fe,gra_fe = PP.param(seq.seq)

    fickett_fe = fickett.fickett_value(seq.seq)
    hexamer = FrameKmer.kmer_ratio(seq.seq,6,3,coding,noncoding)
    Len,Cov,inte_fe = len.len_cov(seq.seq)
    
    tem = [inte_fe,Cov,insta_fe,T2,C0,PI_fe,Len,AC,T0,G0,C2,A4,G2,TG,A0,TC,G1,C3,T3,A1,GC,T1,G4,C1,G3,A3,gra_fe,hexamer,C4,AG,fickett_fe,A2,T4,C,G,A,T]
    feature.write("\t".join(map(str,[seqid,inte_fe,Cov,insta_fe,T2,C0,PI_fe,Len,AC,T0,G0,C2,A4,G2,TG,A0,TC,G1,C3,T3,A1,GC,T1,G4,C1,G3,A3,gra_fe,hexamer,C4,AG,fickett_fe,A2,T4,C,G,A,T]))+"\n")
 
    # print (tmp)
    print (out_label)
    for label,item in enumerate(tem):
        print (tmp)
        print(str(label+1)+':'+str(item))

tmp.close()

# predicting

In [22]:
range_file ='CPPred/Human_Model/Human.range'
model_file ='CPPred/Human_Model/Human.model'

os.system('CPPred/libsvm-3.22/svm-scale -r '+ range_file + ' test.f_svm  > test.scaled ')
os.system('CPPred/libsvm-3.22/svm-predict -b 1 test.scaled ' + model_file +' tmp.txt >  tmp2.txt')

coding_poten = open('coding_potential','w')
coding_poten.write("\t".join(map(str,["table","coding_potential"]))+"\n")

23

In [23]:
lines = open('tmp.txt').readlines()
for line in lines:
    print(line)

labels 1 -1



In [24]:
for line in lines:
    if line[0]=="l":
        continue
    coding_potential=line.split(" ")[1]
    if line.split(" ")[0] == "1":
        coding_poten.write("\t".join(map(str,["coding",coding_potential]))+"\n")
    else:
        coding_poten.write("\t".join(map(str,["noncoding",coding_potential]))+"\n")

In [25]:
output_file ='Human_coding.result'
os.system("paste test.feature coding_potential >"  + output_file)

0

In [14]:
os.system("rm test.*")
os.system("rm coding_potential")
os.system("rm tmp*")

0

# result

In [34]:
lines = open(output_file).readlines()
for line in lines[:2]:
    print(line+'\n')

#ID	ORF-integrity	ORF-coverage	Instability	T2	C0	PI	ORF-length	AC	T0	G0	C2	A4	G2	TG	A0	TC	G1	C3	T3	A1	GC	T1	G4	C1	G3	A3	Gravy	Hexamer	C4	AG	Fickett	A2	T4	C	G	A	T	


gi|1238280249|ref|NM_001354928.1|	1	0.5685152947028103	39.78766097240482	0.5450248756218905	0.0004975124378109452	5.14666748046875	2286	0.10923115202786762	0.00845771144278607	0.0002487562189054726	0.47960199004975124	1.0	0.4557213930348259	0.1435680517541677	0.003233830845771144	0.11644687733266981	0.19577114427860698	0.7231343283582089	0.8002487562189055	0.2942786069651741	0.1042547897486937	0.3	0.9945273631840796	0.21019900497512436	0.7149253731343284	0.7803482587064676	-0.32982917214191904	-0.4564206520992873	0.9930348258706467	0.15501368499626772	0.9186000000000001	0.527363184079602	0.996268656716418	0.24353233830845772	0.2878109452736318	0.2407960199004975	0.22786069651741295	


